This will eventually be an introduction to numerical methods in partial differential equations. Over time, I hope to develop a resource which can be used to learn the theory and practice of numerical PDEs. The philosophy of this text is to be fully rigorous while connecting theory to practice with frequent coded examples of numerical solvers.

The essential prerequisites to be able to effectively use this text are calculus, linear algebra, and some coding knowledge. The first portion of this text is about finite difference methods, which are in effect carefully chosen Taylor approximations. Most other things can be picked up while reading.

The theory of numerical PDEs cannot exist apart from PDEs. Numerical schemes which work well for one type of PDE may fail miserably when adapted to even the simplest PDE of a different type. More importantly, knowledge of how the PDE can be solved and its solution informs the design of a numerical method to approximate its solution. PDEs with known exact solutions can be good test beds for numerical methods.

Let's start with an overview of some common PDEs. I make an attempt to use $t$ to represent a time-like variable and $x$ to represent a space-like variable. If there are three dimensions in particular, may use $\mathbf x = (x, y, z)$ I try to use a bold $\mathbf x$ to represent an $n$-dimensional space-like variable. I say time-like and space-like because there is mathematically no difference between the letters $x$ and $t$, but the nature of PDEs is such that the behavior of the solution may be qualitatively very different in the directions of different variables. The letters $u$ and $v$ commonly denote solutions to a PDE and are understood to be functions of $x$ and $t$. The partial derivative operators may be written as $\frac{\partial}{\partial x}$ or just $\partial_x$. The partial derivatives of $u$ are written as $u_x$ and $u_t$. Second derivatives are written similarly: $u_{tx} = (u_t)_x = \partial_x \partial_t u$. We freely assume that all solutions are sufficiently smooth that mixed partial derivatives commute, i.e., that $u_{xt} = u_{tx}$, etc. Note that there are [well-known examples](https://en.wikipedia.org/wiki/Symmetry_of_second_derivatives) in which this is not true.

The heat equation in one dimension: $u_t + au_{xx} = 0$, where $a > 0$ is a positive constant. The function $u$ models the evolution of temperature in a one-dimensional medium of constant thermal diffusivity $a$.

The heat equation in three dimensions: $u_t + a(u_{xx} + u_{yy} + u_{zz}) = 0$.

The wave equation: $u_{tt} = c^2(u_{xx} + u_{yy} + u_{zz})$.

The sum of all spacial second derivatives, $u_{xx} + u_{yy} + u_{zz}$, is common enough that it is given a dedicated symbol: $\Delta u$. So we can rewrite the heat equation as $u_t + a \Delta u = 0$ and the wave equation as $u_{tt} = c^2 \Delta u$. 

Poisson's equation: $\Delta u = f(\mathbf x)$, where $f$ is a function which may be specified. There is no time-like variable in this equation. If $f$ is just $0$, we get Laplace's equation: $\Delta u = 0$.

Black-Scholes equation: $v_t + \frac 1 2 \sigma^2 s^2 v_{ss} + r s v_s - r v = 0$, where $v$ is the price of a financial option on some stock, $s$ is the price of that stock, $r$ is the risk-free interest rate, and $\sigma$ is the volatility of the stock. This example emphasizes that PDEs need not model physical phenomena. 


The heat equation, the wave equation, and Laplace's equation all have significant physical importance and are archetypes for the three basic types of PDEs: parabolic, hyperbolic, and elliptic. Rather than define them, which takes some work, it is useful to describe them by their qualities. In parabolic equations, such as the heat equation, information travels with infinite speed. If our domain is the real interval $[-100, 100]$, then a change at the initial condition $u(-50, 0)$ for $x = -50$ will change the solution far away (say, $x = 50$) instantly: for any positive time $t > 0$, the solution $u(50, t)$ will be different than it would be without the change.

This is in stark contrast with the wave equation, in which information always travels with speed exactly $c$. If $c=1$, then a change at $x=-50$ can't be felt at $x=50$ until time $t = 100$. The behavior of information propagating at finite speed is the key feature of hyperbolic equations, such as the wave equation.

Lastly, elliptic PDEs (such as Poisson's equation and Laplace's equation) can be thought of as equations in which there is no time-like variable. They may represent the state of a system which is in equilibrium, i.e., not changing in time. This connection may be seen from the similarity between the heat equation and Laplace's equation. If we take the heat equation ($u_t + a \Delta u = 0$) and impose that the solution be in equilibrium, we get the condition that $u$ does not change in $t$, i.e., $u_t = 0$. If we make this substitution, we get the equation $a \Delta u = 0$, which is just Laplace's equation, and we see how in this case, the elliptic Laplace's equation may be thought of as a steady-state solution to the parabolic heat equation.

For the first portion of this text, we will focus on solutions to parabolic PDEs such as the heat equation. We will use the lowercase letter $u$ to refer to true solutions to a PDE and the capital letter $U$ to indicate solutions to a numerical scheme. Taylor's formula is our primary tool to come up with a scheme. Remember that Taylor's formula tells us that if $f$ is a function which is at least $k+1$ times continuously differentiable at a point $x_0$, then $$f(x_0 + h) = f(x_0) + f'(x_0)h + f''(x_0) \frac {h^2}{2!} + \cdots + f^{(k)}(x_0) \frac{h^k}{k!} + R(h).$$
Notice that this is an equality, not an approximation. The uncertainty comes from not knowing what $R(h)$ is. Lagrange's form for the error tells us that 
$$R(h) = f^{(k+1)}(\xi_h) \frac{h^{k+1}}{(k+1)!},$$
where $\xi_h$ is some number between $x$ and $x+h$. There is still uncertainty here because we have no idea what $\xi_h$ is, only that it lies between $x$ and $x+h$. But if we know that $f^{(k+1)}(\xi)$ isn't too big for *any* $\xi$, then it doesn't matter what $\xi_h$ is. 


We made the assumption that $f^{(k+1)}(x)$ is continuous, so if the domain of our PDE is nice ([compact](https://en.wikipedia.org/wiki/Extreme_value_theorem)), then we know that $f^{(k+1)}(x)$ is bounded on the domain, that is, there is some number $M$ for which $|f^{(k+1)}(x)| \leq M$ for all $x$ in the domain. Then we can bound $|R(h)| \leq \frac M {(k+1)!} h^{k+1}$. In practice, we don't know or care how big $\frac M {(k+1)!}$ is, only that it is finite.


This brings us to the standard notation for tracking asymptotic errors: [big O notation](https://en.wikipedia.org/wiki/Big_O_notation). Big O notation is often used to indicate that one function doesn't grow any faster than another function, and thus we can ignore all but the most quickly growing term. It is most commonly used to compare functions as the inputs go to infinity (for example, $x^2 = O(e^x)$ as $x$ goes to $\infty$), but we will use it as inputs go to $0$. We will also be using it for functions which may have more than one input. 

Formally, for functions $f(x, y)$ and $g(x, y)$, we say that $f$ is *big O* of $g$ (as $x$ and $y$ go to $0$), and write $f = O(g)$ or $f(x, y) = O(g(x, y))$ if there is a constant $C > 0$ such that whenever $x$ and $y$ are close enough to $0$, we have $|f(x, y)| \leq C g(x, y)$. The language "whenever $x$ and $y$ are close enough to $0$" is imprecise as written, and means that there exists some values $x_0$ and $y_0$ such that whenever $|x| < x_0$ and $|y| < y_0$, the statement $|f(x, y)| \leq C g(x, y)$ is true. This may be modified in a clear way for functions which have only one or more than two inputs.

In this text, we will never use big O to measure the growth of functions as their inputs go to infinity, so we will suppress the language "as $x$ and $y$ go to $0$." Some examples will be illustrative:
- $\sqrt{x^2 + y^2} = O(1)$ because if $|x| \leq 1$ and $|y| \leq 1$, then $\sqrt{x^2 + y^2} \leq \sqrt{1 + 1} \leq 2$. Note that "$1$" may be thought of as the function whose value is always $1$. 
- $5x^3 = O(x^3)$ because if $|x| \leq 1$ then $5 x^3 \leq 6 (x^3)$. This illustrates that we really need to be able to put a constant in front of the function inside of $O(\cdot)$, or else this could never work.
- $x^3= O(x^2)$. This is the example to pay attention to. If $|x| < 1$, then $|x^3|/x^2 = |x| < 1$, and we see that $x^3$ is smaller than $x^2$.
- $\Delta x^2 + \Delta x^3 + \Delta t + \Delta t^3= O(\Delta x^2 + \Delta t)$. When we are working with powers of the inputs, it is always the smallest power of that variable that is the limiting factor. 

This allows us to precisely guarantee the accuracy of a Taylor approximation. For a second order approximation to a function $f$ with continuous third derivative, we have that $f(x+h) = f(x) + f'(x)h + f''(x)h^2/2 + R(h)$, where $R(h) = f'''(\xi_h) h^3/6$ for some $\xi_h$ between $x$ and $x+h$. We don't know exactly what $f'''(\xi_h)$ is, but we know that it is bounded by some constant $M$ (this uses the continuity of $f'''$ and compactness of the domain), and so $|R(h)| \leq h^3 M/6$. That is to say, $R(h) = O(h^3)$. This is a general fact about Taylor approximations: assuming the domain is nice and the derivatives are smooth, the $k$-th order Taylor approximation to $f(x+h)$ (which uses up to the $k$-th derivative of $f$) has error $O(h^{k+1})$. 